# Need to organize these functions
## Currently just a code dump

In [ ]:
def get_counts(table):
    new_var = pd.DataFrame({'LF_CNT_BUREAU_RECORD': df_bureau.groupby('SK_ID_CURR')['SK_ID_CURR'].count()})
    table = pd.merge(table, new_var, on='SK_ID_CURR', suffixes=('',''))
    return table

In [ ]:
for col in col_to_transform:
    new_var = df_bureau.groupby(['SK_ID_CURR', col]).count()[['SK_ID_BUREAU']].reset_index().pivot(index='SK_ID_CURR',
                                                                                                    columns=col,
                                                                                                    values='SK_ID_BUREAU')
    names = []
    for col_name in new_var.columns:
        name = 'LF_CNT_' + col + '_' + col_name
        names.append(name)
    new_var.columns = names
    new_var.fillna(0, inplace=True)
    df_application = pd.merge(df_application, new_var, on='SK_ID_CURR', suffixes=('',''))

In [ ]:
# Number of Categories
def categorical_counts(data, col_to_transform, table_name, df_ids):
    new_data = df_ids
    for col in col_to_transform:
        new_var = data.groupby(['SK_ID_CURR', col]).count()[['SK_ID_PREV']].reset_index().pivot(index='SK_ID_CURR',
                                                                                                        columns=col,
                                                                                                        values='SK_ID_PREV')
        names = []
        for col_name in new_var.columns:
            name = f'LF_CNT_{col}_{col_name.replace(" ",  "_")}_{table_name}'
            names.append(name)
        new_var.columns = names
        new_var.fillna(0, inplace=True)
        new_data = pd.merge(df_ids, new_var, on='SK_ID_CURR', suffixes=('', ''))
    return new_data

In [ ]:
# Statistics around Numerical values conditional on 'CREDIT_ACTIVE'
def get_num_var_stats(data, meta_data, df_ids):
    new_data = df_ids
    new_vars = data[[meta_data['foreign_key']]+meta_data['num_vars']] \
                .groupby(['SK_ID_CURR']) \
                .aggregate([np.max,
                            np.min, 
                            np.median, 
                            np.mean, 
                            np.std]).reset_index()
    names = []
    for cat in new_vars.columns.levels[0].drop('SK_ID_CURR'):
        for agg in new_vars.columns.levels[1].drop(''):
            name = [f'{cat}_{agg}_{meta_data["table_name"]}']
            names = names + name
    names = names + ['SK_ID_CURR']
    new_vars.columns = names
    new_data = pd.merge(new_data, new_vars, on='SK_ID_CURR')
    return new_data

In [ ]:
def get_binary_var_counts(data, meta_data, df_ids):
    new_data = df_ids
    new_vars = data[[meta_data['foreign_key']]+meta_data['binary_vars']] \
                .groupby(['SK_ID_CURR']) \
                .count() \
                .reset_index()
    new_data = pd.merge(new_data, new_vars, on="SK_ID_CURR")
    return new_data